In [35]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.linalg import qr
import pickle
from tqdm import tqdm
import pywt
#from random import randint

BAD = ['VT', 'VFb', 'VFt'] # KIS,S
project_dir = r'C:\Users\jodge\Documents\School\Summer24\tinymlcontest2022_demo_example'
dir_path = r'C:\Users\jodge\Documents\School\Summer24\tinyml_contest_data_training'
directory_files = os.listdir(dir_path)

training_script = os.path.join(project_dir, 'training_save_deep_models.py')
testing_script = os.path.join(project_dir, 'testing_performances.py')
models_dir = os.path.join(project_dir, 'models')
results_dir = os.path.join(project_dir, 'results')

"""
Energy captured: ranks needed
 5%: 14  10%: 29  15%: 46  20%: 64   25%:  84
30%: 106 35%: 129 40%: 154 45%: 182  50%:  215
55%: 252 60%: 297 65%: 354 70%: 443  75%:  552
80%: 670 85%: 798 90%: 935 95%: 1084 100%: 1250 
"""

'\nEnergy captured: ranks needed\n 5%: 14  10%: 29  15%: 46  20%: 64   25%:  84\n30%: 106 35%: 129 40%: 154 45%: 182  50%:  215\n55%: 252 60%: 297 65%: 354 70%: 443  75%:  552\n80%: 670 85%: 798 90%: 935 95%: 1084 100%: 1250 \n'

In [36]:
def readObj(filename):
    print(f"reading from {filename}")
    with open(filename, 'rb') as file:
       return pickle.load(file)
def saveObj(obj, filename):
    print(f"saving to {filename}")
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)

In [37]:
def setCorrectModel(path, wanted = "original"):
    if wanted == "original":
        #os.rename(os.path.join(path, 'model_1.py.og'), os.path.join(path,'model_1.py'))
        return f"ren {os.path.join(path, 'model_1.py.og')} model_1.py"
    elif wanted == "avgpool":
        #os.rename(os.path.join(path, 'model_1.py.avgpool'), os.path.join(path,'model_1.py'))
        return f"ren {os.path.join(path, 'model_1.py.avgpool')} model_1.py"
        
def resetModel(models_dir, used = "original"):
    if used == "original":
        #os.rename(models_dir, os.path.join(models_dir, 'model_1.py.og'))
        return f"ren {os.path.join(models_dir, 'model_1.py')} model_1.py.og"
    elif used == "avgpool":
        #os.rename(models_dir, os.path.join(models_dir,'model_1.py.avgpool'))
        return f"ren {os.path.join(models_dir, 'model_1.py')} model_1.py.avgpool"
    else:
        print("BAD!!!!!!!!!!!!!")

In [38]:
def prepareData(): #for SVD
    problemSetupsDir = r'C:\Users\jodge\Documents\School\Summer24\playground\data'
    problemSetup = readObj(os.path.join(problemSetupsDir, 'full.pkl'))
    print(problemSetup.r, problemSetup.p) # make sure it's working
        
    for fi in tqdm(directory_files): #special iterator that makes a progress bar
        temp = np.loadtxt(os.path.join(dir_path,fi))
        mse, recon = measureAndReconstruct(problemSetup, temp)
        trunc = problemSetup.C @ temp
        
        outReconName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\recon_data', 'R' + fi) 
        outTruncName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\trunc_data', 'T' + fi)
        np.savetxt(outReconName, recon, fmt='%.7f')
        np.savetxt(outTruncName, trunc, fmt='%.7f')
    
    print("DONE")
    return r, p # ive been using p (which should be r+1) for file naming. idk why i chose that convention.

In [39]:
class SVD:
    def __init__(self, U, S, VT):
        self.U = U
        self.S = S
        self.VT = VT

class ProblemSetup:
  def __init__(self, r, p, fullSVD, truncSVD, Q, R, pivots, C):
      self.r = r # ranks to use
      self.p = p # number of sensors
      self.fullSVD = fullSVD
      self.truncSVD = truncSVD
      self.Q = Q
      self.R = R
      self.pivots = pivots
      self.C = C
      
def getFullTrainSignalMatrix(): #all files with each as a column
    pkl_name = r"fullTrainMat.pkl"
    pkl_loc = os.path.join("data", pkl_name)

    if os.path.exists(pkl_loc):
        answer = readObj(pkl_loc)
        return answer
    else:
        print(f"{pkl_loc} not found - see other notebooks for generation")
        
def processData(trainingData, label, r = 400, p = 500): # r = ranks desired, p = num sensors. bug when r=p, math is hard.
    U, S, VT = svd(trainingData)
    full_SVD = SVD(U, S, VT)
    
    #reshape SVD according to r
    U_hat, S_hat, VT_hat = U[:,:r], S[:r,:r], VT[:r,:]
    trunc_SVD = SVD(U_hat, S_hat, VT_hat)
    
    Q, R, pivots = None, None, None
    if (p == r):
        Q, R, pivots = qr(U_hat, pivoting = True) # or maybe just U
    elif (p > r): # oversampled
        Q, R, pivots = qr(U_hat @ U_hat.T, pivoting = True) # or maybe just U
    else:
        for _ in range(100):
            print("ERROR p < r")
            
    pivots = pivots[:p]
    
    # Create C matrix
    C = np.zeros((p, getFullTrainSignalMatrix().shape[0]))
    #print(C.shape, pivots.shape)
    C[np.arange(p), pivots] = 1

    problemSetup = ProblemSetup(r, p, full_SVD, trunc_SVD, Q, R, pivots, C)
    filename = os.path.join("data", label + ".pkl")
    saveObj(problemSetup, filename)
    
    return problemSetup
    
def svd(x = getFullTrainSignalMatrix()):
    U, S, VT = np.linalg.svd(x, full_matrices=True) #full_matrices=False
    S = np.diag(S)
    return (U, S, VT)

def measureAndReconstruct(problemSetup, signal):
    # Measure a signal
    C, U_hat, p, r, pivots =  problemSetup.C, problemSetup.truncSVD.U, problemSetup.p, problemSetup.r, problemSetup.pivots
    y = C @ signal
    
    # Solve for coefficients
    U_k_reduced = U_hat[:, :p][pivots, :]
    
    if p == r: # broken, bad shapes
        a = np.linalg.lstsq(U_k_reduced, y, rcond=None)[0]
    else:
        a = np.linalg.pinv(C @ U_hat) @ y

    x_reconstructed = U_hat @ a
    mseFinal = np.mean((signal - x_reconstructed) ** 2)
    return mseFinal, x_reconstructed

reading from data\fullTrainMat.pkl


In [40]:
def batGen(rp, numRuns=5, mode = "normal", sampleRate = 1.0, length = 1250, model = "original", kept = 100):
    ans = []

    recon_indices_dir = os.path.join(project_dir, 'recon_indices\\')
    trunc_indices_dir = os.path.join(project_dir, 'trunc_indices\\')
    drop_indices_dir = os.path.join(project_dir, 'drop_indices\\')
    fft_indices_dir = os.path.join(project_dir, 'fft_indices\\')
    
    data_dir = r'C:\Users\jodge\Documents\School\Summer24'
    recon_data_dir = os.path.join(data_dir, 'recon_data\\')
    trunc_data_dir = os.path.join(data_dir, 'trunc_data\\')
    drop_data_dir = os.path.join(data_dir, 'drop_data\\')
    fft_data_dir = os.path.join(data_dir, 'fft_data\\')

    path_flags = ''
    if mode == 'recon':
        path_flags = f'--path_data {recon_data_dir} --path_indices {recon_indices_dir}'
    elif mode == 'trunc':
        path_flags = f'--path_data {trunc_data_dir} --path_indices {trunc_indices_dir}'
    elif mode == 'drop' or mode == 'random':
        path_flags = f'--path_data {drop_data_dir} --path_indices {drop_indices_dir}'
    elif mode == 'fft':
        path_flags = f'--path_data {fft_data_dir} --path_indices {fft_indices_dir}'
        

    size_flags = ''
    if length < 98:
        length = 98
    if length != 1250:
        if model == 'avgpool':
            size_flags = f'--size {length}'

    sample_flags = f'--sample_rate {sampleRate}' if sampleRate != 1.0 else ''
    sample_flags_temp = f'_{int(sampleRate)}' if sample_flags else ''
    sample_flags_temp.replace('.','-')
    
    temp_kept_flag = '_k'+str(kept) if kept != 100 else ''
    print(sample_flags_temp, temp_kept_flag)

    out_train_name = f'train_{mode}_r{rp}_{length}{sample_flags_temp}_{model}.txt'
    out_test_name = f'TEST_{mode}_r{rp}_{length}{sample_flags_temp}{temp_kept_flag}_{model}.csv'
    output_train_file = os.path.join(results_dir, out_train_name)
    output_test_file = os.path.join(results_dir, out_test_name)

    temp_train_out = 'train_temp.txt'
    temp_test_out = 'test_temp.txt'
    train_command = f'\tpython {training_script} {path_flags} {sample_flags} {size_flags} >> {output_train_file}'
    test_command = f'\tpython {testing_script} {path_flags} {sample_flags} {size_flags} >> {output_test_file}'

    loop_head = r'for /l %%i in (1,1,' + str(numRuns) + r') do ('
    loop_tail = ')'

    ans.append(r'@echo off')
    ans.append(setCorrectModel(models_dir, model))  
    ans.append(loop_head)
    
    ans.append('\techo iteration %%i')
    ans.append(train_command)
    ans.append(test_command)
    
    ans.append(loop_tail)
    ans.append(resetModel(models_dir, model))
    ans.append('pause')
    
    bat_name = out_test_name.split(".")[0]
    bat_name = "_".join(bat_name.split("_")[1:]) + ".bat"

    bat = os.path.join("bats", bat_name)
    with open(bat,"w") as output:
        for line in ans:
            output.write(line + '\n')
    
    return ans, bat_name 

In [41]:
def batGenNew(out_train_name, out_test_name, numRuns = 5, indices_folder = None, data_folder=None, length=1250, model = 'original', additional_flags = ''):
    ans = []

    output_train_file = os.path.join(results_dir, out_train_name)
    output_test_file = os.path.join(results_dir, out_test_name)

    path_flags = ''
    if data_folder:
        data_path = os.path.join(r'C:\Users\jodge\Documents\School\Summer24', data_folder)
        path_flags = path_flags + f' --path_data {data_path}'
    if indices_folder:
        indices_path = os.path.join(project_dir, indices_folder)
        path_flags = path_flags + f' --path_indices {indices_path}'
    #path_flags = f'--path_data {data_path} --path_indices {indices_path}' if indices_folder or data_folder else ''

    size_flags = ''
    if length < 98:
        length = 98
    if length != 1250 and model == 'avgpool':
        size_flags = f'--size {length}'

    train_command = f'\tpython {training_script} {path_flags} {size_flags} {additional_flags} >> {output_train_file}'
    test_command = f'\tpython {testing_script} {path_flags} {size_flags} {additional_flags} >> {output_test_file}'

    loop_head = r'for /l %%i in (1,1,' + str(numRuns) + r') do ('
    loop_tail = ')'

    ans.append(r'@echo off')
    ans.append(setCorrectModel(models_dir, model))  
    ans.append(loop_head)
    
    ans.append('\techo iteration %%i')
    ans.append(train_command)
    ans.append(test_command)
    
    ans.append(loop_tail)
    ans.append(resetModel(models_dir, model))
    ans.append('pause')
    
    bat_name = out_test_name.split(".")[0]
    bat_name = "_".join(bat_name.split("_")[1:]) + ".bat"

    bat = os.path.join("bats", bat_name)
    with open(bat,"w") as output:
        for line in ans:
            output.write(line + '\n')
    
    return bat_name 

# CHOOSE WHAT TO RUN FROM HERE!
### Set your variables and then move the .bat to the right folder and then run it from CMD.

# SVD

In [27]:
r = 1250
p = r#+1
fullTrainSignalMatrix = getFullTrainSignalMatrix()
problemSetup = processData(fullTrainSignalMatrix, "full", r, p)
r0,p0 = prepareData()
print(r,r0, p,p0) # sanity check

reading from data\fullTrainMat.pkl
reading from data\fullTrainMat.pkl
saving to data\full.pkl
reading from C:\Users\jodge\Documents\School\Summer24\playground\data\full.pkl
1250 1250


100%|██████████████████████████████████████████████████████████████████████████| 30213/30213 [3:32:06<00:00,  2.37it/s]


DONE
1250 1250 1250 1250


In [31]:
ans, bat_name = batGen(p, mode = "trunc", sampleRate = 1.0, length = p, model="avgpool")
print(bat_name)

 
trunc_r1250_1250_avgpool.bat


# Random Dropout (Static Mask)

In [ ]:
def random_drop_mask(keep_percentage, length=1250):
    """ CLAUDE
    Randomly drop data points from a NumPy array.
    
    :param data: Input NumPy array
    :param keep_percentage: Percentage of data points to keep (0-100)
    :return: Filtered NumPy array with the desired percentage of data points
    """
    if not 0 < keep_percentage <= 100:
        raise ValueError("Keep percentage must be between 0 and 100")
    
    # Generate a binary mask with the desired probability of keeping each element
    keep_mask = np.random.binomial(1, keep_percentage / 100, size=length).astype(bool)
    
    # Apply the mask to the original data
    #filtered_data = data[keep_mask]
    
    return keep_mask
    
def randomDropout(keep_percent = 50, trunc = True):
    mask = random_drop_mask(keep_percent)
    print(f"keep: {keep_percent}%. mask: {np.sum(mask)}") 

    for fi in tqdm(directory_files, desc="Static Random Mask"): #special iterator that makes a progress bar
        temp = np.loadtxt(os.path.join(dir_path,fi))
        dropped = None
        if trunc:
            dropped = temp[mask]
        else:
            dropped = np.where(mask, temp, 0)
        
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\drop_data', 'D' + fi)
        np.savetxt(outName, dropped, fmt='%.7f')
    
    print("DONE")
    return np.sum(mask) # how many data points were kept (should be close to 1250 * keep percent)


In [ ]:
keeping = 8.56 #kep 50%
trunc = False
mod = "original"
if mod == "original":
    trunc = False
masklen = randomDropout(keeping, trunc)
length = masklen if trunc else 1250    

In [ ]:
keeping = str(keeping).replace('.','-')  
ans, bat_name = batGen(masklen, mode = "drop", length = length, model = "original", kept = keeping)
print(bat_name)

# Random Dropout

In [ ]:
keep_percentage = 50
longest = 0
model = 'original'
trunc = True
if model == 'original':
    trunc = False

for fi in tqdm(directory_files[:], desc=f"Totally Random {keep_percentage}%"): #special iterator that makes a progress bar
    temp = np.loadtxt(os.path.join(dir_path,fi))
    temp_mask = np.random.binomial(1, keep_percentage / 100, size=1250).astype(bool)
    temp_len = np.sum(temp_mask)
    longest = temp_len if temp_len > longest else longest
    
    if trunc:
        dropped = temp[temp_mask]
    else:
        dropped = np.where(temp_mask, temp, 0)
        
    outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\drop_data', fi)
    np.savetxt(outName, dropped, fmt='%.7f')

keep_percentage = str(keep_percentage).replace('.','-')
out_train_name = f'train_rand_k{keep_percentage}_l{longest}_{model}.txt'
out_test_name = f'TEST_rand_k{keep_percentage}_l{longest}_{model}.csv'
data_folder = 'drop_data\\'
bat_name = batGenNew(out_train_name, out_test_name, numRuns = 5, indices_folder = None, data_folder=data_folder, length=longest, model = 'avgpool')
bat_name

# FFT

In [43]:
def fftMask(ranks = 250):
    pkl_name = "fftMask" + str(ranks) + ".pkl"
    pkl_loc = os.path.join("data", pkl_name)
    mask = None

    if os.path.exists(pkl_loc):
        mask = readObj(pkl_loc)
    else:
        temp = np.fft.rfft(np.loadtxt(os.path.join(dir_path,directory_files[0])))
        counts = np.zeros_like(temp)
        print()
        for fi in tqdm(directory_files, desc = f"Generating FFT mask of {ranks} ranks."):
            temp = np.loadtxt(os.path.join(dir_path,fi))
            fft = np.fft.rfft(temp)
            
            ranked = np.argsort(np.abs(fft))[::-1]
            top_indices = ranked[:ranks]
            counts[top_indices] += 1
        #print(f"counts {counts}")
        
        plt.plot(counts)
        plt.title("counts")
        plt.show()
            
        final_indices = np.argsort(counts)[-ranks:]
        #print(f"{len(final_indices)} final idxs: {final_indices}")
        mask = np.zeros_like(counts, dtype=bool)
        mask[final_indices] = True
        saveObj(mask, pkl_loc)
        
    print(f"mask of len {len(mask)} ready")
    return mask

def fftSparseGen(mask, ranks, trunc = False):
    for fi in tqdm(directory_files, desc="Files Processing"):
        temp = np.loadtxt(os.path.join(dir_path,fi))
        fft = np.fft.rfft(temp)

        if trunc:
            done = fft[:][mask]
        else: # zeroes otherwise
            done = np.where(mask, fft, 0) 
        #print(len(done))
        
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\fft_data', 'F' + fi)
        np.savetxt(outName, done.real, fmt='%.7f')
    
    print("DONE")

In [46]:
bandpass_mask = np.zeros(626, dtype=bool)
pass_band_filter = list(range(15*5,55*5))
bandpass_mask[pass_band_filter] = True

ranks = len(pass_band_filter)
truncate = True
#fftSparseGen(fftMask(ranks), ranks, trunc = truncate)
fftSparseGen(bandpass_mask, ranks, trunc = truncate)

Files Processing: 100%|█████████████████████████████████████████████████████████| 30213/30213 [00:46<00:00, 642.93it/s]

DONE


In [47]:
len_temp = ranks if truncate else 1250
ans, bat_name = batGen(ranks, mode = "fft", length = len_temp, model = "avgpool")
print(bat_name)

 
fft_r200_200_avgpool.bat


# Wavelet

In [48]:
def basicDWTProcess(w='db4', m='zero'):
    data = np.loadtxt(os.path.join(dir_path,directory_files[1]))
    cA, cD = pywt.dwt(data, wavelet=w, mode=m)
    size = len(cA)
    
    for fi in tqdm(directory_files, desc="Files Processing"):
        data = np.loadtxt(os.path.join(dir_path,fi))
        cA, cD = pywt.dwt(data, wavelet=w, mode=m)
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\temp_data', fi)
        np.savetxt(outName, cA, fmt='%.7f')
    
    return size #return length after processing
    
def waveletDecomp(w='db4', m='zero', l=5, r=3):
    data = np.loadtxt(os.path.join(dir_path,directory_files[1]))
    coeffs = pywt.wavedec(data, w, mode=m, level=l)
    #print(coeffs[r])
    size = len(coeffs[r])
    
    for fi in tqdm(directory_files, desc=f"{w} {m} {l}"):
        data = np.loadtxt(os.path.join(dir_path,fi))
        coeffs = pywt.wavedec(data, w, mode=m, level=l)
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\temp_data', fi)
        np.savetxt(outName, coeffs[r], fmt='%.7f')
    
    return size #return length after processing  

### simple DWT

In [49]:
w = 'db12'
m = 'zero'
length = basicDWTProcess(w,m)

mod = 'avgpool'
out_train_name = f'train_dwt_{w}_{m}_r{length}_l{length}_{mod}.txt'
out_test_name = f'TEST_dwt_{w}_{m}_r{length}_l{length}_{mod}.csv'

data_folder = "temp_data\\"

bat_name = batGenNew(out_train_name, out_test_name, numRuns = 5, data_folder=data_folder, model = mod, length=length)
bat_name

Files Processing: 100%|█████████████████████████████████████████████████████████| 30213/30213 [01:13<00:00, 408.65it/s]


'dwt_db12_zero_r636_l636_avgpool.bat'

### configurable wavelet decomp

In [18]:
w = 'coif3' # wavelet_list = ['db4', 'db6', 'db8', 'sym4', 'sym6', 'sym8', 'coif3', 'coif4', 'bior3.3', 'bior3.5']

m = 'zero' # zero symmetric periodic
levels = 5
wanted = 3 #informed by waveletMaskCounter, seems to always be 3 at level 5 decomp
length = waveletDecomp(w,m,l=levels,r=wanted)

size = 98 if length < 98 else length

mod = 'original'
w = w.replace('.','-')
file_setup = f'{w}_{m}_{levels}-{wanted}_r{length}-{size}_{mod}'
out_train_name = f'train_{file_setup}.txt'
out_test_name = f'TEST_{file_setup}.csv'

data_folder = "temp_data\\"

bat_name = batGenNew(out_train_name, out_test_name, numRuns = 10, data_folder=data_folder, model = mod, length=length)
bat_name

coif3 zero 5: 100%|█████████████████████████████████████████████████████████████| 30213/30213 [03:18<00:00, 152.53it/s]


'coif3_zero_5-3_r171-171_original.bat'

# Short Time Length

### note that on a model (avgpool) that accepts shorter than 1250 length inputs, you can just use the --size arg when running (hence "trunc" will be False by default)

In [12]:
def shortenFiles(length, trunc = False):
    
    for fi in tqdm(directory_files, desc=f"Shortening Data {length}/1250"):
        data = np.loadtxt(os.path.join(dir_path,fi))
        data = data[:length]
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\temp_data', fi)
        np.savetxt(outName, data, fmt='%.7f')

In [19]:
mod = 'original'
points = 107

#additional_flags = ''
if mod == 'original':
    length = 1250
else:
    length = points
    #additional_flags = f'-- size {length}'

shortenFiles(points, False)

out_train_name = f'train_short_{points}_{length}_{mod}.txt'
out_test_name = f'TEST_short_{points}_{length}_{mod}.csv'

data_folder = "temp_data\\"

bat_name = batGenNew(out_train_name, out_test_name, numRuns = 5, data_folder=data_folder, model = mod, length=length, additional_flags=additional_flags)
bat_name

Shortening Data: 100%|██████████████████████████████████████████████████████████| 30213/30213 [00:41<00:00, 727.29it/s]


'short_107_1250_original.bat'

# Final Idea
### Half Sample Rate, Half Length, DWT db4 zero cA

In [62]:
def assemblage(sample_rate = 2, length_percent = 625/1250, wavelet = 'db12', padding = 'zero'):
    size = 0
    for fi in tqdm(directory_files, desc=f"Assemblage"):
        data = np.loadtxt(os.path.join(dir_path,fi))
        n = len(data)
        
        data = data[:int(n * length_percent)]
        data = data[::sample_rate]
        cA, cD = pywt.dwt(data, wavelet=wavelet, mode=padding)
        
        size = len(cA)
        outName = os.path.join(r'C:\Users\jodge\Documents\School\Summer24\temp_data', fi)
        np.savetxt(outName, cA, fmt='%.7f')
    
    return size #return length after processing

In [63]:
mod = 'avgpool'

sample_rate = 2
length_percent = 625/1250
wavelet='db4'
padding='zero'
epochs = 2

length = assemblage()


Assemblage: 100%|███████████████████████████████████████████████████████████████| 30213/30213 [03:14<00:00, 155.25it/s]


In [64]:
if mod == 'original':
    length = 1250

out_train_name = f'train_assemblage_sr{sample_rate}_len{length_percent*100}_{wavelet}_{padding}_cA_fs{length}_{mod}.txt'
out_test_name = f'TEST_assemblage_sr{sample_rate}_len{length_percent*100}_{wavelet}_{padding}_cA_fs{length}_{mod}.csv'

data_folder = "temp_data\\"
additional_flags = '' if epochs == 2 else f'--epoch {epochs}'

bat_name = batGenNew(out_train_name, out_test_name, numRuns = 10, data_folder=data_folder, model = mod, length=length, additional_flags=additional_flags)
bat_name

'assemblage_sr2_len50.bat'